In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [8]:
# Create Spark session
spark = SparkSession.builder \
    .appName("ReadCSVExample") \
    .getOrCreate()


In [9]:
(
    # Read the CSV file located at the specified path, treating the first row as header
    spark.read.csv("./sample.csv", header=True)

    # Add a new column called 'new_column'
    # If 'old_column' > 10, assign 10; otherwise assign 0
    .withColumn(
        "new_column", F.when(F.col("old_column") > 10, 10).otherwise(0)
    )

    # Filter rows where 'old_column' is greater than 8
    .where("old_column > 8")

    # Group the data by the values in 'new_column'
    .groupby("new_column")

    # Count how many rows exist per 'new_column' group
    .count()

    # Write the resulting grouped data to a CSV file
    # Overwrite the file if it already exists
    .write.csv("updated_frequencies.csv", mode="overwrite")
)



Why PySpark Output is a Folder with part-* Files
- Distributed System:
PySpark (Spark) is designed to run on distributed clusters. When your job runs, each executor processes a partition of the data. Each partition's result is written independently into a file.

- Parallelism:
The files like part-00000.csv, part-00001.csv, etc., correspond to individual partitions of the DataFrame, written by different workers in parallel. These are saved inside the specified output directory.

- Scalability:
Writing multiple files is efficient and scalable for big data workflows. It avoids single-node bottlenecks when writing huge files.

- Metadata Files:
You may also see files like _SUCCESS, which indicate the job completed successfully.

What to Do If You Want a Single File
If you're working locally or with small data and want to get one CSV file, you can:

Option 1: Coalesce to 1 partition

```python
df.coalesce(1).write.csv("output_single_file.csv", mode="overwrite", header=True)
```
Warning: This forces all data into one partition — not scalable for large datasets.

Option 2: Use .toPandas().to_csv() (for small datasets)
```python
df.toPandas().to_csv("single_file.csv", index=False)
```

## Example: Merge part-*.csv into One File

In [11]:
import os
import pandas as pd

# Path to the Spark output directory
input_folder = "updated_frequencies.csv"
output_file = "merged_output.csv"

# Get all part files (ignore _SUCCESS or hidden files)
part_files = [f for f in os.listdir(input_folder) if f.startswith("part-") and f.endswith(".csv")]

# Read and concatenate all part files
df_list = [pd.read_csv(os.path.join(input_folder, f)) for f in part_files]
merged_df = pd.concat(df_list)

# Save to a single CSV file
merged_df.to_csv(output_file, index=False)

print(f"Merged into: {output_file}")

Merged into: merged_output.csv
